# Information retrieval --- Module 1: Evaulation
Santhosh Kumar Rajamanickam, Dennis Ulmer and Stian Steinbakken

TODO: Maybe insert a small introduction here? 


## Theoretical part [15pts]
### 1 Hypothesis Testing --- The problem of multiple comparisons. 
How many hypothesis tests, m, does it take to get to (with Type I error for each test = α):
1. P(mth experiment gives significant result | m experiments lacking power to reject H0)?
2. P(at least one significant result | m experiments lacking power to reject H0)?



**Answer**

1. Because the outcome of the experiments are independent, we can just multiply the probabilities of the outcomes in this question. The probability of an experiment giving a significant result, i.e. rejecting the Null Hypothesis, will be $\alpha$. Likewise, the probability of experiments lacking the power to reject $H_0$ will be $1-\alpha$. Therefore

\begin{equation}
P(m^{th}\text{ experiment gives significant result } | \ m\text{ experiments lacking power to reject } H_0) = (1-\alpha)^{m-1}\alpha
\end{equation}

2. The probability of at least one in $m$ experiments rejecting the Null Hypothesis falsely is the sum of the probabilities of a experiment at time step $t$ rejecting the Null-Hypothesis conditioned on the previous experiments rejecting it correctly:

\begin{equation}
   p(\text{rejecting the Null Hypothesis at least once}|m\text{ experiments lacking power to reject }H_0)\\ = p(1^{st}\text{ wrong}) + p(2^{nd} \text{wrong}\ |\ 1^{st}\ \text{right}) + \ldots + p(m^{th} \text{wrong}\ |\ \text{all}\ m-1\ \text{experiments}\ \text{right})\\
   = \alpha + (\alpha - 1)\alpha + (\alpha-1)^2\alpha + \ldots + (\alpha-1)^{m-1}\alpha\\
   = \alpha \cdot {\displaystyle \sum_{t=0}^{m-1}} (\alpha-1)^t
\end{equation}

### 2 Bias and unfairness in Interleaving experiments [10 points]
Balance interleaving has been shown to be biased in a number of corner cases. An example was given during the lecture with two ranked lists of length 3 being interleaved, and a randomly clicking population of users that resulted in algorithm A winning ⅔ of the time, even though in theory the percentage of wins should be 50% for both algorithms. Can you come up with a situation of two ranked lists of length 3 and a distribution of clicks over them for which Team-draft interleaving is unfair to the better algorithm?


**Answer**

Assume we have to algorithms A and B providing documents for an interleaved ranking of length $3$, where A is the worse algorithm. While constructing the interleaved ranking, the result of two coin tosses where A wins gives A preference to place its documents first. Because we only have three entries, two of the entries will originate from A and only one from B.

Given random clicking, A will again win ⅔ of times. Here the unfairness is due to the coin tosses.

## Experimental part [85 pts] 
Step 1: Simulate Rankings of Relevance for E and P (5 points)
In the first step you will generate pairs of rankings of relevance, for the production P and experimental E, respectively, for a hypothetical query q. Assume a 3-graded relevance, i.e. {N, R, HR}. Construct all possible P and E ranking pairs of length 5. This step should give you about.

Example:
P: {N N N N N}
E: {N N N N R}
…
P: {HR HR HR HR R}
E: {HR HR HR HR HR}


In [1]:
# Step 1 code

from itertools import product
from collections import namedtuple

RankingPair = namedtuple("RankingPair", ["E", "P"])

RANKINGS = ("N", "R", "HR")

simulations = []

# Create all possible rankings of length 5
for ranking_pair in list(product(list(product(RANKINGS, repeat=5)), repeat=2)):
    simulations.append(RankingPair(*ranking_pair))

print("{} simulations in total.\n".format(len(simulations)))
    
print("First 10 simulations:\n")
for ranking_pair in simulations[:10]:
    print("P: {}, E: {}".format(*ranking_pair))
    
print("\nLast 10 simulations:\n")
for ranking_pair in simulations[-10:]:
    print("P: {}, E: {}".format(*ranking_pair))


59049 simulations in total.

First 10 simulations:

P: ('N', 'N', 'N', 'N', 'N'), E: ('N', 'N', 'N', 'N', 'N')
P: ('N', 'N', 'N', 'N', 'N'), E: ('N', 'N', 'N', 'N', 'R')
P: ('N', 'N', 'N', 'N', 'N'), E: ('N', 'N', 'N', 'N', 'HR')
P: ('N', 'N', 'N', 'N', 'N'), E: ('N', 'N', 'N', 'R', 'N')
P: ('N', 'N', 'N', 'N', 'N'), E: ('N', 'N', 'N', 'R', 'R')
P: ('N', 'N', 'N', 'N', 'N'), E: ('N', 'N', 'N', 'R', 'HR')
P: ('N', 'N', 'N', 'N', 'N'), E: ('N', 'N', 'N', 'HR', 'N')
P: ('N', 'N', 'N', 'N', 'N'), E: ('N', 'N', 'N', 'HR', 'R')
P: ('N', 'N', 'N', 'N', 'N'), E: ('N', 'N', 'N', 'HR', 'HR')
P: ('N', 'N', 'N', 'N', 'N'), E: ('N', 'N', 'R', 'N', 'N')

Last 10 simulations:

P: ('HR', 'HR', 'HR', 'HR', 'HR'), E: ('HR', 'HR', 'R', 'HR', 'HR')
P: ('HR', 'HR', 'HR', 'HR', 'HR'), E: ('HR', 'HR', 'HR', 'N', 'N')
P: ('HR', 'HR', 'HR', 'HR', 'HR'), E: ('HR', 'HR', 'HR', 'N', 'R')
P: ('HR', 'HR', 'HR', 'HR', 'HR'), E: ('HR', 'HR', 'HR', 'N', 'HR')
P: ('HR', 'HR', 'HR', 'HR', 'HR'), E: ('HR', 'HR', 'HR', 'R

### Step 2: Implement Evaluation Measures (10 points)
Implement 1 binary and 2 multi-graded evaluation measures out of the 7 measures mentioned above. 


In [2]:
# Step 2 code
# TODO: Dennis: implement 1 multi-graded
# TODO: Santosh: implement 1 multi-graded

def precision(rankings):
    relevant_tags = (RANKINGS[1], RANKINGS[2]) # 'R' and 'HR'
    
    relevant_rankings = sum([1 for rank in rankings if rank in relevant_tags])
    return relevant_rankings / len(rankings) 

# For our binary evaulation meassure we implement average precision (AP)
def average_precision(rankings):
    relevant_tags = (RANKINGS[1], RANKINGS[2]) # 'R' and 'HR'
    return sum([precision(rankings[:i+1]) for i in range(len(rankings)) if rankings[i] in relevant_tags]) / len(rankings)

def test_average_precision():
    example_rank_1 = ('R', 'R', 'R', 'R', 'R')
    example_rank_2 = ('N', 'N', 'N', 'N', 'N')
    example_rank_3 = ('N', 'R', 'N', 'N', 'R')
    """
    Example rank 3 should yield (1/2 + 2/5) / 5 = (0.5 + 0.4) / 5 = 0.18
    """
    
    assert average_precision(example_rank_1) == 1
    assert average_precision(example_rank_2) == 0
    assert average_precision(example_rank_3) == 0.18
    return True

AP_passed = test_average_precision()
if AP_passed: print("All average-precision tests passed")


All average-precision tests passed


In [3]:
# Implementation of Normalized Discounted Cumulative Gain at rank k (nDCG@k)
import math

# Assumptation
RANKING_WEIGHTS = {"N": 0, "R": 1, "HR": 5}

def ndcg(ranks, k=5):
    return sum([(2**RANKING_WEIGHTS[rank]-1) / math.log(2+i, 2) for i, rank in enumerate(ranks[:k])])

def evaluate(ranking_pairs, metric, **metric_args):
    scores = []
    
    for ranking_pair in ranking_pairs:
        scores.append((metric(ranking_pair.E, **metric_args), metric(ranking_pair.P, **metric_args)))
    
    return scores
    

### Step 3: Calculate the 𝛥measure (10 points)
For the three measures and all P and E ranking pairs constructed above calculate the difference: 𝛥measure = measureE-measureP. Consider only those pairs for which E outperforms P.


In [4]:
# Step 3 code
# TODO: Stian

def delta_measure(simulations, measure_to_test):
    # Measure_to_test argument must be a function that takes in a ranking to be measured
    # We do this so that the function can be re-used for each of the measures
    
    delta_m = 0
    for sim in simulations:
        E_ranking, P_ranking = sim
        P_measure, E_measure = measure_to_test(P_ranking), measure_to_test(E_ranking)
        
        # TODO: I am unsure if we are supposed to accumulate the delta, or only consider each simulation by itself
        # Currently accumulating
        delta_m += E_measure - P_measure if E_measure > P_measure else 0
    return delta_m
        
delta_m_AP = delta_measure(simulations, average_precision)
delta_m_ndcg = delta_measure(simulations, ndcg)
print("Delta_m for average_precision:", delta_m_AP)
print("Delta_m for Normalized Discounted Cumulative Gain:", delta_m_ndcg)

# TODO: Run delta_measure for the remaining measures

Delta_m for average_precision: 8967.593333333285
Delta_m for Normalized Discounted Cumulative Gain: 677954.3127262632


### Step 4: Implement Interleaving (15 points)
Implement 2 interleaving algorithms: (1) Team-Draft Interleaving OR Balanced Interleaving, AND (2), Probabilistic Interleaving. The interleaving algorithms should (a) given two rankings of relevance interleave them into a single ranking, and (b) given the users clicks on the interleaved ranking assign credit to the algorithms that produced the rankings.

(Note 4: Note here that as opposed to a normal interleaving experiment where rankings consists of urls or docids, in our case the rankings consist of relevance labels. Hence in this case (a) you will assume that E and P return different documents, (b) the interleaved ranking will also be a ranking of labels.)

In [5]:
# Step 4 code
# TODO: Stian (I can do both)
from random import choice 

def team_draft_interleaving(rankings_1, rankings_2, interleaved_rankings_size=5):
    interleaved_rankings = []
    i = -1
    while len(interleaved_rankings) < interleaved_rankings_size:
        i += 1
        
        if choice(["H", "T"]) == "H":
            interleaved_rankings.append(rankings_1[i])
            
            if len(interleaved_rankings) < interleaved_rankings_size:
                interleaved_rankings.append(rankings_2[i])
        else:
            interleaved_rankings.append(rankings_2[i])
            
            if len(interleaved_rankings) < interleaved_rankings_size:
                interleaved_rankings.append(rankings_1[i])
            
    return interleaved_rankings

example_rank_1 = ('R', 'R', 'R', 'R', 'R')
example_rank_2 = ('N', 'N', 'N', 'N', 'N')

interleaved = team_draft_interleaving(example_rank_1, example_rank_2)
print(interleaved)
        

['R', 'N', 'R', 'N', 'R']


### Step 5: Implement User Clicks Simulation (15 points)
Having interleaved all the ranking pairs an online experiment could be ran. However, given that we do not have any users (and the entire homework is a big simulation) we will simulate user clicks.

We have considered a number of click models including:
1. Random Click Model (RCM)
2. Position-Based Model (PBM)
3. Simple Dependent Click Model (SDCM)
4. Simple Dynamic Bayesian Network (SDBN)

Consider two different click models, (a) the Random Click Model (RCM), and (b) one out of the remaining 3 aforementioned models. The parameters of some of these models can be estimated using the Maximum Likelihood Estimation (MLE) method, while others require using the Expectation-Maximization (EM) method. Implement the two models so that (a) there is a method that learns the parameters of the model given a set of training data, (b) there is a method that predicts the click probability given a ranked list of relevance labels, (c) there is a method that decides - stochastically - whether a document is clicked based on these probabilities.



In [39]:
import codecs


class YandexData:
    """ Class to represent the entire dataset. """
    def __init__(self, sessions=[]):
        self.sessions = sessions
        
    def add(self, session):
        self.sessions.append(session)
        
    def __getitem__(self, items):
        return self.sessions[items]
        
    def __iter__(self):
        return (session for session in self.sessions)
    
    def __len__(self):
        return len(self.sessions)
    
    @property
    def queries(self):
        """ Return all queries in the entire data set. """
        for session in self:
            for query in session.queries:
                yield query
                
    @property
    def clicks(self):
        """ Return all clicks in the entire data set. """
        for session in self:
            for click in session.clicks:
                yield click
                
    def __repr__(self):
        return "<YandexData comprising {} sessions>".format(len(self))


class Session:
    def __init__(self, uid, user_actions):
        self.uid = uid
        self.user_actions = user_actions
        
    def __iter__(self):
        return (user_action for user_action in self.user_actions)
    
    @property
    def queries(self):
        """ Return all queries in this session. """
        return (action for action in self.user_actions if action.action_type == "Q")
    
    @property
    def clicks(self):
        """ Return all clicks in this session. """
        return (action for action in self.user_actions if action.action_type == "C")
    
    def __repr__(self):
        return "<Session #{}>".format(self.uid)
        

class UserAction:
    def __init__(self, session_id, time_passed, action_type, *document_ids):
        self.session_id = session_id
        self.time_passed = time_passed
        self.action_type = action_type
        self.document_ids = document_ids
        
    def __iter__(self):
        return (document_id for document_id in self.document_ids)
            
    def __contains__(self, document_id):
        return document_id in self.document_ids
        
class Query(UserAction):
    def __init__(self, session_id, time_passed, serp_id, query_id, *document_ids):
        self.query_id = query_id
        self.serp_id = serp_id
        super().__init__(session_id, time_passed, "Q", serp_id, *document_ids)
        
    def __repr__(self):
        return "<Query #{} in session #{}>".format(self.query_id, self.session_id)
        
        
class Click(UserAction):
    def __init__(self, session_id, time_passed, *document_ids):
        super().__init__(session_id, time_passed, "C", *document_ids)
    
    def __repr__(self):
        return "<Click in session #{} on #{}>".format(self.session_id, self.document_ids)
        


def read_yandex_data(path="./YandexRelPredChallenge.txt"):
    """ Parse the data set into an appropriate data structure. """
    data = YandexData()
    current_session_id = 0
    current_actions = []
    
    with codecs.open(path, "rb", "utf-8") as file:
        for line in file.readlines():
            columns = line.strip().split("\t")
            columns = tuple(map(lambda x: int(x) if x not in ("Q", "C") else x, columns))  # Parse numbers
            session_id, time_passed, action_type, *document_ids = columns
            
            # New session, create object, save it and start a new one
            if session_id != current_session_id:
                session = Session(current_session_id, current_actions)
                data.add(session)
                
                current_session_id = session_id
                current_actions = []
            
            if action_type == "Q":
                # Get query id
                query_id, serp_id, *document_ids = document_ids
                current_actions.append(Query(session_id, time_passed, serp_id, query_id, *document_ids))
            else:
                # It's a click!
                current_actions.append(Click(session_id, time_passed,  *document_ids))
                
    return data
        
yandex_data = read_yandex_data()
print(yandex_data, "\n")

# You can now iterate over sessions in the data set like this:
for session in yandex_data:
    pass  # Do something

# You iterate over slices of the data set:
for session in yandex_data[5:10]:
    print(session)
    
print("")
    
# You can iterate over all the queries in the entire data set:
for query in yandex_data.queries:
    pass  # Do something

# ... or of a single session
for query in yandex_data[0].queries:
    print(query)
    
# (Multiple queries with the same id might be the user clicking trough result pages?)
    
print("")
    
for click in yandex_data[0].clicks:
    print(click)

<YandexData comprising 11716 sessions> 
<Session #5>
<Session #6>
<Session #7>
<Session #8>
<Session #9>

<Query #0 in session #0>
<Query #0 in session #0>
<Query #0 in session #0>
<Query #0 in session #0>
<Query #0 in session #0>

<Click in session #0 on #()>
<Click in session #0 on #()>
<Click in session #0 on #()>


In [7]:
# Step 5 code
# TODO: Santhosh: implement RCM
# TODO: Dennis: one of the remaining 3

In [ ]:
class ClickModel:
    
    def simulate_clicks(ranking):
        """ Let the model determine the clicks given a ranking. """
        pass

class PositionBasedModel(ClickModel):
    def __init__(self, click_prob, examination_prob, attractiveness_prob):
        p

### Step 6: Simulate Interleaving Experiment (10 points)
Having implemented the click models, it is time to run the simulated experiment.

For each of interleaved ranking run N simulations for each one of the click models implemented and measure the proportion p of wins for E.
(Note 7: Some of the models above include an attractiveness parameter 𝑎uq. Use the relevance label to assign this parameter by setting 𝑎uq for a document u in the ranked list accordingly. 

In [ ]:
# TODO: Stian

### Step 7: Results and Analysis (30 points)
Compare the results of the offline experiments (i.e. the values of the 𝛥measure) with the results of the online experiment (i.e. proportion of wins), analyze them and reach your conclusions regarding their agreement.


In [ ]:
# Step 7 code and markdown
# TODO: we should probably all contribute to this part